In [2]:
import aria2p
import csv
import cv2
import glob
import hashlib
import uuid
import time
import os

from PIL import Image

In [3]:
aria2 = aria2p.API(
    aria2p.Client(
        host="http://localhost",
        port=6800,
        secret=""
    )
)

## aria2c

``aria2c --enable-rpc --connect-timeout=5 --max-connection-per-server=16 --timeout=5 --max-concurrent-downloads=1024``

In [4]:
DOWN_PER_SEC = 64
SLEEP = 1 / DOWN_PER_SEC

delimiter = ";"

filenames = glob.glob("data/csv/*.csv")

skip_urls = []
skip_urls.append("/assets/cat-ads.png")
skip_urls.append("/assets/cat-browser.png")

N = len(filenames)
for i in range(0, N):
    filename = filenames[i]
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=delimiter, quotechar='|')
        print(f"{i + 1}|{len(filenames)}, {filename}")
        header = next(reader)
        for i, (url, label, subcategory, categories) in enumerate(reader):
            try:
                if url in skip_urls:
                    print(url)
                    continue
    
                file_id = url + subcategory + categories
                file_id = str(int(hashlib.sha1(file_id.encode("utf-8")).hexdigest(), 16) % (10 ** 8))
                file_id = file_id + "_" + str(uuid.uuid4())
                save_path = f"data/scrolller/img/{file_id}.jpg"
                download = aria2.add(url, {"out": save_path})
                time.sleep(SLEEP)
            except Exception as e:
                print(i, subcategory, categories, e)

1|71, data/csv/data_00010_00e6005a-4967-469a-a836-a6e1a23519ea.csv
/assets/cat-ads.png
2|71, data/csv/data_00038_c22e9402-0275-4e1b-94e9-b1ccd7cc5ffd.csv
/assets/cat-ads.png
/assets/cat-browser.png
3|71, data/csv/data_00024_149ff635-9cf6-4907-9d7e-28f7ef888888.csv
/assets/cat-ads.png
/assets/cat-browser.png
4|71, data/csv/data_00015_6c8174a7-a7e0-46a6-9fb3-8f62567acce5.csv
/assets/cat-ads.png
/assets/cat-browser.png
5|71, data/csv/data_00029_02ee8d8d-4cff-44f5-9105-b2ea17203fc2.csv
/assets/cat-ads.png
6|71, data/csv/data_00004_3f3b567a-7f49-482c-9e58-bf9e54ee4530.csv
/assets/cat-ads.png
/assets/cat-browser.png
7|71, data/csv/data_00021_61f52f56-39f9-437b-841d-d1caacf4cbfa.csv
/assets/cat-ads.png
/assets/cat-browser.png
2 likeus ['animals', 'none'] No URI to download.
8|71, data/csv/data_00005_e9aaac08-6e11-49db-8897-27e2c5db644d.csv
/assets/cat-ads.png
/assets/cat-browser.png
9|71, data/csv/data_00025_c93d7054-42ad-4fe9-b1be-b511cbf41005.csv
/assets/cat-ads.png
/assets/cat-browser.png


In [5]:
BASE_PATH = "/home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller"
img_path = os.path.join(BASE_PATH, f"img")
img_filenames = glob.glob(os.path.join(img_path, "*"))
img_filenames = sorted(img_filenames)
LEN = len(img_filenames)
for i in range(0, LEN):
    img_filename = img_filenames[i]

    if (i % 1000) == 0:
        print(i, LEN, img_filename)

    check_img = os.path.exists(img_filename)

    if "aria" in img_filename:
        print(i, LEN, img_filename, "SKIP: aria")
        continue
        
    try:
        if check_img:
            img = cv2.imread(img_filename)
            
            if img is None:
                print(i, LEN, img_filename, "REMOVE: None")
                os.remove(img_filename)
                continue
                
            if len(img.shape) == 2:
                print(i, LEN, img_filename, "REMOVE: Gray")
                os.remove(img_filename)
                continue
            
            img = Image.open(img_filename)
            img = img.convert('RGB')
            if 'icc_profile' not in img.info:
                continue
            img.info.pop('icc_profile')
            img.save(img_filename)

    except Exception as e:
        print(img_filename, e)

0 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/10000169_3e368c4e-8bc8-4497-99e5-f867f498a68f.jpg
1000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/10092339_fd8252a9-d3f4-40fd-873e-7a1c18131743.jpg


/home/henning/tmp/experiments/ai/stable_diffusion/venv/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


2000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/10187775_f9071505-ab0c-4d87-be04-f96088ca66ac.jpg
3000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/10284239_538fcfa2-cda2-4df6-bd6f-7b167de0a308.jpg
4000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/10386898_2cfaa029-03b1-407d-ba04-18f18429280c.jpg


libpng warning: bKGD: invalid
libpng warning: bKGD: invalid


5000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/10485782_d468819c-9815-4dd3-8e79-61c36f832c09.jpg
6000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/10587729_37f28482-16e9-4fca-be28-b0121a428d86.jpg
7000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/10687508_903ce45b-5d5c-4a0a-a58c-a29739e22ba7.jpg
8000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/10788771_50acffb8-7169-46d5-8eb3-a8dd9cfc2af6.jpg


...

917000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/97894321_35885d38-63c1-4347-a069-4833af05337d.jpg
918000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/97992466_fae43be2-6fa1-4232-bc5b-6a9c7b36ddb3.jpg
919000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/980833_9b6b44cf-001e-4f74-9d45-49d8f8da7ed4.jpg
920000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/981769_bdc86808-b510-4d16-9702-83271b75ed28.jpg
921000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/98276668_076fb89d-3495-4b2d-adbc-abc97f5110f2.jpg
922000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/9836571_5f9feec7-cb3f-41a2-9924-dbf3d8080c5d.jpg
922039 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/98369851_0debe31f-f21f-4ce5-abd1-b55616cd0ac5.jpg REMOVE: None
923000 939229 /home/henning/tmp/experiments/ai/stable_diffusi

libpng warning: iCCP: known incorrect sRGB profile


925000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/98649095_ec18226b-1dc9-4d5f-a323-6bf02a952e86.jpg
926000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/98745515_9cedab8f-3bb1-4519-85fb-7e6af2b1d94c.jpg
927000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/98834645_1fb65b60-ef66-4157-ae9d-cca473e59152.jpg
928000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/98933043_889ebeda-d0ec-4622-89b0-5cbfacd9818b.jpg
929000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/9902181_505c98b7-bf49-486e-a89e-b67c8c013300.jpg
930000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/9911190_472631ac-f57a-4fce-ba64-e4cd7f9d2db0.jpg
931000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/99204469_9eb4b361-3c9a-4e26-a5ab-9572046a1b3a.jpg
932000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/sc

libpng warning: iCCP: known incorrect sRGB profile


935000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/99592892_73595e7a-ba98-4313-b295-cb9d7fc71f55.jpg
936000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/99685397_a37ce9d9-1bc7-4c8a-8a3a-1d31f56d2c25.jpg
937000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/99785947_e608fb5e-e9a7-4817-9985-bcfad100f475.jpg
938000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/99886142_a41a5f8b-6e72-43bc-8317-73cc24c526ef.jpg
939000 939229 /home/henning/tmp/experiments/ai/stable_diffusion/data/scrolller/img/99980999_194d5bbf-0e32-4b3b-9ae9-3c1783317437.jpg
